# CNNで画像認識

## ライブラリの読み込み

In [1]:
# データ処理のライブラリ
import numpy as np 
import pandas as pd

# データ可視化のライブラリ
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

# 機械学習ライブラリ scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import fetch_mldata

# 機械学習ライブラリ keras
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


## データセットの読み込み

In [2]:
# MNISTデータセットの読み込み
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

# 訓練・テストデータの結合
X = np.append(X_train, X_test, axis=0)
X = X.reshape(X.shape[0], 784)
y = np.append(y_train, y_test, axis=0)

# train_test_splitを使い再分割
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=42) 

# 訓練データの最初の1万件のみへ縮小
X_train = X_train[:10000]
y_train = y_train[:10000]

# テストデータの最初の1万件のみへ縮小
X_test = X_test[:10000]
y_test = y_test[:10000]

# 正規化
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

11493376/11490434 [==============================] - 0s 0us/step


## データの前処理

In [3]:
# 特徴量データのサイズ
print(X_train.shape)
print(X_test.shape)

(10000, 784)
(10000, 784)


In [4]:
# Keras用にデータを整形
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)
print(X_train.shape)
print(X_test.shape)

(10000, 28, 28, 1)
(10000, 28, 28, 1)


In [5]:
# ターゲットデータの確認
print(y_train[0:5])
print(y_test[0:5])

[5 4 8 0 2]
[8 4 8 7 7]


In [6]:
# ターゲットデータをクラスベクトルへ変換
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

print(y_train[0:5])
print(y_test[0:5])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


## モデルの構築

In [7]:
# Sequentialモデル
model = Sequential()

W0811 07:39:15.412032 139809815730048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [8]:
# 入力層と畳み込み層
model.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))

W0811 07:39:17.378441 139809815730048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 07:39:17.396696 139809815730048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [9]:
# プール層
model.add(MaxPooling2D(pool_size=(2, 2)))

W0811 07:39:18.361875 139809815730048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Dropoutは、意図的にニューラルネットワークのユニットをランダムに 0 とする。  
Dropoutにより、過学習の防止に役立つ。  

Dropoutの引数 0.25 は、ユニット全体の25%をランダムに 0 とすることを意味する。  
どのような割合にするのかは、CNNの構造・データの特性により様々。  
一般的には、0.25 or 0.5 が使用される。

In [10]:
# ドロップアウト
model.add(Dropout(0.25))

W0811 07:39:19.656718 139809815730048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0811 07:39:19.675464 139809815730048 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# 全結合層
# １つ前の層から出力されたデータを平らなベクトルにする
model.add(Flatten())
# 隠れ層を追加
model.add(Dense(128, activation='relu')) 

In [0]:
# 出力層
model.add(Dense(10, activation='softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1352)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               173184    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 174,554
Trainable params: 174,554
Non-trainable params: 0
_________________________________________________________________


## モデルのコンパイル

In [14]:
# モデル コンパイル
# loss: 損失関数
# optimizer: 最適化アルゴリズム
# metrics: 評価関数
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

W0811 07:39:27.090183 139809815730048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0811 07:39:27.115196 139809815730048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



## モデルの訓練

In [15]:
# モデル訓練
model.fit(X_train, y_train, epochs=50)

W0811 07:39:29.238267 139809815730048 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
10000/10000 [==============================] - 8s 759us/step - loss: 1.8750 - acc: 0.4570
Epoch 2/50
10000/10000 [==============================] - 1s 111us/step - loss: 0.6416 - acc: 0.8132
Epoch 3/50
10000/10000 [==============================] - 1s 109us/step - loss: 0.4793 - acc: 0.8596
Epoch 4/50
10000/10000 [==============================] - 1s 108us/step - loss: 0.4222 - acc: 0.8699
Epoch 5/50
10000/10000 [==============================] - 1s 108us/step - loss: 0.3784 - acc: 0.8852
Epoch 6/50
10000/10000 [==============================] - 1s 108us/step - loss: 0.3466 - acc: 0.8940
Epoch 7/50
10000/10000 [==============================] - 1s 108us/step - loss: 0.3245 - acc: 0.9009
Epoch 8/50
10000/10000 [==============================] - 1s 109us/step - loss: 0.2972 - acc: 0.9103
Epoch 9/50
10000/10000 [==============================] - 1s 108us/step - loss: 0.2751 - acc: 0.9157
Epoch 10/50
10000/10000 [==============================] - 1s 128us/step - loss: 0.2655 - a

## 分類

In [0]:
# テストデータで推測結果を出力
cnn_pred = model.predict(X_test)

In [17]:
print(cnn_pred[0])

[1.1773145e-08 6.0070960e-08 4.6360379e-05 1.1352274e-03 7.8738260e-09
 4.2698835e-07 1.6903282e-08 6.3102257e-09 9.9881774e-01 1.1111147e-07]


In [0]:
# 推測結果をクラスラベルへ変換
y_pred_CNN = np.argmax(cnn_pred, axis=1)
y_test_CNN = np.argmax(y_test, axis=1)

In [19]:
print(confusion_matrix(y_test_CNN,y_pred_CNN))

[[ 973    0    1    0    0    0    4    2    1    2]
 [   1 1127   11    2    0    0    2    2    2    5]
 [   1    3  945    3    3    0    0    2    9    1]
 [   1    0   17  983    1    5    1    5   11   10]
 [   2    0    2    0  876    0    4    2    1   19]
 [   0    1    2   20    0  896    6    0   10    2]
 [   3    2    1    0    2    5  946    0    2    0]
 [   1    0   17    1    3    1    0 1014    2   16]
 [   6    2   10   10    2    2    2    3  924    8]
 [   5    3    0    9    4    1    0    5    6 1003]]


In [20]:
print(accuracy_score(y_test_CNN,y_pred_CNN))

0.9687
